In [ ]:
import sys
import os

import numpy as np
import pandas as pd
import pickle
from tqdm import tnrange, tqdm_notebook

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import autosklearn.classification


In [ ]:
#To load from pickle file
path = path = "/home/conor/legal/autoML/data/features/{}"

features = []
with open(path.format('features.pickle'), 'rb') as fr:
    features = pickle.load(fr)
    fr.close()
print(len(features))
features[0]

In [ ]:
def autoML(filename):
    
    path = "/home/conor/legal/autoML/data/features/{}.csv"
    row = filename.split("_")

    train = pd.read_csv(path.format("train_"+filename))
    X_train = train.drop(columns='target')
    y_train = train['target']
    
    test = pd.read_csv(path.format("test_"+filename))
    X_test= test.drop(columns='target')
    y_test = test['target'] 
    
    automl = autosklearn.classification.AutoSklearnClassifier(
        ensemble_size=1,
        initial_configurations_via_metalearning=0,
        time_left_for_this_task=120,
        per_run_time_limit=30,
        resampling_strategy='cv',
        resampling_strategy_arguments={'folds': 10},
        include_preprocessors=["no_preprocessing"],
        n_jobs = 6
    )

    # fit() changes the data in place, but refit needs the original data. We
    # therefore copy the data. In practice, one should reload the data
    automl.fit(X_train.copy(), y_train.copy(), dataset_name='legal_jazz')

    automl.refit(X_train.copy(), y_train.copy())
    
    model = automl.show_models()
    stats = automl.sprint_statistics()
    predictions = automl.predict(X_test)
    
    train_accuracy = float(stats.split('Best validation score: ')[1].split("\n")[0])
    test_accuracy = sklearn.metrics.accuracy_score(y_test, predictions)
    
    row = row + [train_accuracy,test_accuracy,model,stats,predictions]
    
    return row


In [ ]:
row = autoML("ngram_2000_2_both_none")

In [ ]:
results = []
for f in tqdm_notebook(features[0:5]):
    try:
        result = autoML(f)
        results.append(result)
    except: 
        print(f)
    
print(len(results))